# Abstract
#### Setting : 실험목적, 실험에 사용한 데이터셋 명시

                            1. 원시데이터인 VOC.xlsx  데이터를 사용
                            2. VOC 유형을 Label로 지정
                            3. 사용하지 않는  컬럼제거
                            5. 특수 문자 제거
                            6. Kospacing 라이브러리를 이용하여 1차 자동 띄어 쓰기 라이브러리 적용
                            7. 정교한 띄어쓰기를 위해 han-spell라이브러리를 이용하여 2차 자동띄어쓰기 적용 
                            8. 불용어 처리

                                                      
                           8666 rows × 4 columns

In [1]:
import pandas as pd
import sys

In [2]:
from collections import Counter

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os


import warnings
warnings.filterwarnings('ignore')
import timeit

In [3]:
from pykospacing import Spacing
spacing = Spacing()
spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")
"김형호 영화시장 분석가는 '1987'의 네이버 영화 정보 네티즌 10점 평에서 언급된 단어들을 지난해 12월 27일부터 올해 1월 10일까지 통계 프로그램 R과 KoNLP 패키지로 텍스트마이닝하여 분석했다."
# Apply a list of words that must be non-spacing
spacing('귀밑에서턱까지잇따라난수염을구레나룻이라고한다.')

'귀 밑에서 턱까지 잇따라 난 수염을 구레나 룻이라고 한다.'

In [4]:
df = pd.read_excel('../../Common/data/VOC.xlsx', header=1)
print("train set: {}개".format(len(df)))

train set: 8666개


In [5]:
df.head(3)

,VOC SEQ,접수시간,SAID,코넷ID,서비스 종류,본부,지점,국사,단말종류,펌웨어버전,VOC 유형,세부내용,상품,VOC레벨1유형,고장시설명,고장원인내용,수리내용,최근 3일 장애발생값별 건수,최근 3일 품질발생 건수
0,SR20210927000100-1-J1R-7648,2021-09-27 00:03:15,2.393333e+10,z23933313030,OTS ICOD ACAP,강원고객본부,원주지사,원주국사,DMT-4900HDIP,5.5.2.6,원인불명>조치 전 자연 회복,[고객요청사항]205호tv안됨/상담중자연회복됨/23933327170,비즈형_OTS슬림_일반매장용_ICOD,품질,NaN,NaN,NaN,NaN,0
1,SR20210927000047-1-J1Q-6635,2021-09-27 00:03:30,1.957235e+10,z!19571956740,OTV ICOD UHD,전남고객본부,순천지사,동순천국사,UWAP-101,1.0.0352,실시간 채널>네트워크 오류,"[고객요청사항]601호TV수신불,인터넷연결오류뜸,재부팅해봄,모텔영업용 237일중꼭점...",OTV Biz 슬림_숙박업소용,품질,"실내선(UTP,동CA)",고객부주의(사용미숙),회선정비,NaN,0
2,SR20210927000442-1-GSKO8,2021-09-27 00:06:03,6.106325e+10,z!68063043156,OTS ICOD ACAP,수도권서부고객본부,구로지사,금천국사,NA1110,2.1.3.0,실시간 채널>수신 불량,[고객요청사항]1622호.tv수신불/said61063247367\n[회선/단말특이사...,비즈형_OTS슬림_일반매장용_ICOD,품질,셋탑박스,제품불량(제조사),교체(모뎀/ONT/STB),NaN,0
3,SR20210927000526-1-J1Q-3705,2021-09-27 00:07:13,1.175903e+10,kt657292,OTV ICOD UHD,부산고객본부,창원지사,동마산국사,CT1101,14.00.10,실시간 채널>네트워크 오류,"tv수신불, 네트워크오류, 접수보류",OTV 라이트,품질,NaN,NaN,NaN,"NET-001_NW_LINK_FAIL: 5, VOD-chgE-001_VOD_CHAN...",0
4,SR20210927000846-1-DEZ9JV5,2021-09-27 00:10:51,6.208130e+10,z!63078191862,OTV ICOD ACAP,전남고객본부,순천지사,북여수국사,NA1100,2.04.0180,실시간 채널>수신 불량,[고객요청사항]TV수신불/*연락후~ \n[회선/단말특이사항]\n[진단결과]\n[기타...,OTV 슬림,품질,자연회복,자연회복,NaN,"NET-004_DNS_FAIL: 8, VOD-chgE-001_VOD_CHANGE_F...",0


### 데이터 전처리

In [16]:
from pykospacing import Spacing
spacing = Spacing()

def processsing(df) :
    no_spacing_data = []
    df_col = []
    ## 필요 없는 문자 '호' 제거
    for i in range(len(df['세부내용'])):
        romove_col = df['세부내용'][i].replace('호',"")
        df_col.append(romove_col)
    df['세부내용'] = df_col
    df['세부내용']

    ## 특수문자 제거
    df['세부내용'] = df['세부내용'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    df['sentence'] = df['세부내용']
    df = df['sentence']
    
    
    for i in range(len(df)) :
        try:
            sen = df[i].replace(' ', '')
            no_spacing_data.append(sen)
        except :
            ' '
    df = pd.DataFrame({'세부내용':no_spacing_data})
    return df

### 1차 자동 띄어 쓰기 적용

In [19]:
## 띄어쓰기 로직을 데이터 셋에 적용
def kospacing(df) :
    df = processsing(df)
    clean_data = []
    for row in df.iterrows() :
        
        spacings = spacing(row[1]['세부내용'])
        clean_data.append(spacings)

        preprocess_data = pd.DataFrame({'sentence':clean_data})
        
    return preprocess_data


0.053712초 걸렸습니다.


['고객 요청사 항 안됨상담 중 자연회복됨',
 '고객 요청사항 수신불인터넷 연결오류 뜸재부팅해 봄 모텔 영업용 일중 꼭 점검원하여 지사 이관함회선 단말특이사항 진단 결과 기타 추가 연락처',
 '고객 요청사항 수신불회선 단말특이사항 진단 결과 기타 추가 연락처 연후방',
 '수신 불네트워크 오류 접수 보류',
 '고객 요청사항 수신불 연락 후회선 단말특이사항 진단 결과 기타 추가 연락처인 입',
 '고객 요청사항 채널은 정상 넷플릭스 안됨 포트리셋 미리 연락 후 방문 요망회선 단말특이사항 진단 결과 기타 추가 연락처',
 '고객 요청사항 수신 불서비스 시작 중 회선 단말특이사항 진단 결과 기타 추가 연락처',
 '김현철 채팅지니 자체에서 잡음이 난다고 함고장 접수함 고객 요청사항 셋탑 이상지니 자체에서 따다 다다 거리는 소리가 계속 나다고 함동일 증상으로 일경받았으나 또 소리가 난다고 함재부팅해도 동일 연후방 회선 단말특이사항 진단 결과 기타 추가 연락처',
 '고객 요청사항 나오나 따닥거리는 소리가 난다함리셋 후 잘됨회선 단말특이사항 진단 결과 기타 추가 연락처',
 '고객 요청사항 화질 불량 시청 중 멈추는 증상 반복됨 연락 후회선 단말 특이사항 진단 결과 기타 추가 연락처인 입',
 '고객 요청사항 수신불회선 단말특이사항 진단 결과 기타 추가 연락처 연후방',
 '고객 요청사항 수신불 연락 후 최대한 빠른 방문 요망회선 단말특이사항 진단 결과 기타 추가 연락처',
 '고객 요청사항 채널 정상 넷플릭스만 안됨 포트리셋 연락 후 최대한 빠른 방문 요망회선 단말특이사항 진단 결과 기타 추가 연락처',
 '고객 요청사항 수신불스트림에러 자 연회복회선 단말특이사항 진단 결과 기타 추가 연락처',
 '고객 요청사 항 안됨리셋 후 정상',
 '안심권유 인터넷 지킴이가 입수신불재부팅 안내 후 정상 사용됨',
 '고객 요청사항 넷플릭스 실행 불가리셋 후 잘됨 안심권유회선 단말특이사항 진단 결과 기타 추가 연락처',
 '고객 요청사항 수신불 최대한 빠른 점검 요회선 단말

### 2차 자동 띄어쓰기

In [32]:
from hanspell import spell_checker

## 띄어쓰기 로직을 데이터 셋에 적용
def han_spell(df) :
    clean_data = []
    for i in range(len(df)) :
        spelled_sent = spell_checker.check(df['sentence'][i])
        hanspell_sent = spelled_sent.checked
        clean_data.append(hanspell_sent)
        preprocess_data = pd.DataFrame({'sentence':clean_data})
        
    return preprocess_data


In [ ]:
df = kospacing(df)
preprocess_data = han_spell(df)
data = pd.read_excel('../../Common/data/VOC.xlsx', header=1)

df = pd.concat([data['접수시간'], data['본부'], data['VOC 유형'], preprocess_data['sentence']],axis= 1)

In [60]:
df.head()

,접수시간,본부,VOC 유형,sentence,words_list
0,2021-09-27 00:03:15,강원고객본부,원인불명>조치 전 자연 회복,고객 요청사 항 안됨 상담 중 자연 회복됨,0
1,2021-09-27 00:03:30,전남고객본부,실시간 채널>네트워크 오류,고객 요청사항 수신불인터넷 연결 오류 뜸 재부팅해 봄 모텔 영업용 일중 꼭 점검 원...,0
2,2021-09-27 00:06:03,수도권서부고객본부,실시간 채널>수신 불량,고객 요청사항 수 선불 회선 단말 특이사항 진단 결과 기타 추가 연락처 연후방,0
3,2021-09-27 00:07:13,부산고객본부,실시간 채널>네트워크 오류,수신 불 네트워크 오류 접수 보류,0
4,2021-09-27 00:10:51,전남고객본부,실시간 채널>수신 불량,고객 요청사항 수신 불 연락 우회선 단말 특이사항 진단 결과 기타 추가 연락처인 입,0


### 불용어 처리

In [65]:
stop_words = "고객 요청사항 요청사 항 본인 긴급 독촉심함 독촉 봇 빠른 방문 요청 요청함 중 회선단말특이사항 단말특이사항 불 후 정상 측 연락처 안 됨기사 추가 특이사항 서비스 회선 진단 결과 코로나 기타"

def stop_word(df) :
    df = df.reset_index(drop= True)
    preprocess_data = []
    for i in range(len(df)) :
        preprocess_data.append(df['sentence'][i].split(' '))

    stop_words=stop_words.split(' ')

    df['words_list'] = 0

    result = []
    for sentence_words in preprocess_data:
        list_a = []
        for word in sentence_words:
            if word not in stop_words:
                list_a.append(word)
        result.append(list_a)

    # 리스트를 문자열로 : join 이용
    result1 = " ".join(result[0])

    for i in range(len(result)) :
        result[i] = " ".join(result[i])

    return result

In [68]:
result = stop_word(df)
data = df[['접수시간', '본부', 'VOC 유형']]
data['sentence'] = result

In [69]:
data

,접수시간,본부,VOC 유형,sentence
0,2021-09-27 00:03:15,강원고객본부,원인불명>조치 전 자연 회복,안됨 상담 자연 회복됨
1,2021-09-27 00:03:30,전남고객본부,실시간 채널>네트워크 오류,수신불인터넷 연결 오류 뜸 재부팅해 봄 모텔 영업용 일중 꼭 점검 원하여 지사 이관...
2,2021-09-27 00:06:03,수도권서부고객본부,실시간 채널>수신 불량,수 선불 단말 연후방
3,2021-09-27 00:07:13,부산고객본부,실시간 채널>네트워크 오류,수신 네트워크 오류 접수 보류
4,2021-09-27 00:10:51,전남고객본부,실시간 채널>수신 불량,수신 연락 우회선 단말 연락처인 입
...,...,...,...,...
8661,2021-09-27 23:51:22,충남고객본부,실시간 채널>화질 이상,안심 권유 화질 불량 시청 화면 멈추고 꺼졌다 켜지는 현상이 있다고 함 연락 방문이 단말
8662,2021-09-27 23:52:05,수도권서부고객본부,원인불명>조치 전 자연 회복,리콜 넷플릭스 연령 인증 문의 건 리콜하니 해결했다고 하심
8663,2021-09-27 23:55:04,부산고객본부,실시간 채널>수신 불량,대수 신볼 전송 리셋
8664,2021-09-27 23:57:50,수도권강북고객본부,실시간 채널>네트워크 오류,안심 권유 점검 시청 불이 인터넷 연결 오류나 옴 모든 장비 재부팅 안내 단말


In [70]:
data.to_csv('../dataset/CAI_data_001.csv', index=False)